# How to train your DragoNN tutorial 

## How to use this tutorial

This tutorial utilizes a Jupyter Notebook - an interactive computational enviroment that combines live code, visualizations, and explanatory text. The notebook is organized into a series of cells. You can run the next cell by cliking the play button:
![play button](./tutorial_images/play_button.png)
You can also run all cells in a series by clicking "run all" in the Cell drop-down menu:
![play all button](./tutorial_images/play_all_button.png)
Half of the cells in this tutorial contain code, the other half contain visualizations and explanatory text. Code, visualizations, and text in cells can be modified - you are encouraged to modify the code as you advance through the tutorial. You can inspect the implementation of a function used in a cell by following these steps:
![inspecting code](./tutorial_images/inspecting_code.png)

## Tutorial Overview
In this tutorial, we will:

    1) Simulate regulatory DNA sequence classification task
    2) Train DragoNN models of varying complexity to solve the simulation
    3) Interpret trained DragoNN models
    4) Show how to train your DragoNN on your own, non-simulated data and use it to interpret data

This tutorial is implemented in python (see this [online python course](https://www.udacity.com/course/programming-foundations-with-python--ud036) for an introduction).

We start by loading dragonn's tutorial utilities. Let's review properties of regulatory sequence while the utilities are loading

In [ ]:
%reload_ext autoreload
%autoreload 2
from dragonn.tutorial_utils import *
%matplotlib inline

![sequence properties 1](./tutorial_images/sequence_properties_1.jpg)
![sequence properties 2](./tutorial_images/sequence_properties_2.jpg)
![sequence properties 3](./tutorial_images/sequence_properties_3.jpg)
![sequence properties 4](./tutorial_images/sequence_properties_4.jpg)

In this tutorial, we will simulate heterodimer motif grammar detection task. Specifically, we will simulate a "positive" class of sequences with a SIX5-ZNF143 grammar with relatively fixed spacing between the motifs and a negative class of sequences containing both motifs positioned independently:
![heterodimer simulation](./tutorial_images/heterodimer_simulation.jpg)
Here is an overview of the sequence simulation functions in the dragonn tutorial:
![sequence](./tutorial_images/sequence_simulations.png)

Let's run the print_available_simulations function and see it in action.

In [ ]:
print_available_simulations()

## Getting simulation data

To get simulation data we:
    
    1) Define the simulation parameters
        - obtain description of simulation parameters using the print_simulation_info function
    2) Call the get_simulation_data function, which takes as input the simulation name and the simulation
    parameters, and outputs the simulation data.

We simulate the SIX5-ZNF143 heterodimer motif grammar using the "simulate_heterodimer_grammar" simulation function. To get a description of the simulation parameters we use the print_simulation_info function, which takes as input the simulation function name, and outputs documentation for the simulation including the simulation parameters:

In [ ]:
print_simulation_info("simulate_heterodimer_grammar")

Next, we define parameters for a heterodimer grammar simulation of 500bp long sequence, with 0.4 GC fraction, 10000 positive and negative sequences, with SIx5 and ZNF143 motifs spaced 2-10 bp apart in the positive sequences:

In [ ]:
heterodimer_grammar_simulation_parameters = {
    "seq_length": 500,
    "GC_fraction": 0.4,
    "num_pos": 10000,
    "num_neg": 10000,
    "motif1": "SIX5_known5",
    "motif2": "ZNF143_known2",
    "min_spacing": 2,
    "max_spacing": 10}

We get the simulation data by calling the get_simulation_data function with the simulation name and the simulation parameters as inputs.

In [ ]:
simulation_data = get_simulation_data("simulate_heterodimer_grammar", heterodimer_grammar_simulation_parameters)

simulation_data provides training, validation, and test sets of input sequences X and sequence labels y. The inputs X are matrices with a one-hot-encoding of the sequences:
![one hot encoding](./tutorial_images/one_hot_encoding.png)
Here are the first 10bp of a sequence in our training data:

In [ ]:
simulation_data.X_train[0, :, :, :10]

This matrix represent the 10bp sequence TTGGTAGATA.

Next, we will provide a brief overview of DragoNNs and proceed to train a DragoNN to classify the sequences we simulated:
![classificatioin task](./tutorial_images/classification_task.jpg)

# DragoNN Models

A locally connected linear unit in a DragoNN model can represent a PSSM (part a). A sequence PSSM score is obtained by multiplying the PSSM across the sequence, thersholding the PSSM scores, and taking the max (part b). A PSSM score can also be computed by a DragoNN model with tiled locally connected linear units, amounting to a convolutional layer with a single convolutional filter representing the PSSM, followed by ReLU thersholding and maxpooling (part c).
![dragonn vs pssm](./tutorial_images/dragonn_and_pssm.jpg)
By utilizing multiple convolutional layers with multiple convolutional filters, DragoNN models can represent a wide range of sequence features in a compositional fashion:
![dragonn model figure](./tutorial_images/dragonn_model_figure.jpg)

# Getting a DragoNN model

The main DragoNN model class is SequenceDNN, which provides a simple interface to a range of models and methods to train, test, and interpret DragoNNs. SequenceDNN uses [keras](http://keras.io/), a deep learning library for [Theano](https://github.com/Theano/Theano) and [TensorFlow](https://github.com/tensorflow/tensorflow), which are popular software packages for deep learning.

To get a DragoNN model we:
    
    1) Define the DragoNN architecture parameters
        - obtain description of architecture parameters using the inspect_SequenceDNN() function
    2) Call the get_SequenceDNN function, which takes as input the DragoNN architecture parameters, and outputs a 
    randomly initialized DragoNN model.

To get a description of the architecture parameters we use the inspect_SequenceDNN function, which outputs documentation for the model class including the architecture parameters:

In [ ]:
inspect_SequenceDNN()

"Available methods" display what can be done with a SequenceDNN model. These include common operations such as training and testing the model, and more complex operations such as extracting insight from trained models. We define a simple DragoNN model with one convolutional layer with one convolutional filter, followed by maxpooling of width 35. 

In [ ]:
one_filter_dragonn_parameters = {
    'seq_length': 500,
    'num_filters': [1],
    'conv_width': [45],
    'pool_width': 45}

we get a radnomly initialized DragoNN model by calling the get_SequenceDNN function with one_filter_dragonn_parameters as the input

In [ ]:
import warnings
warnings.filterwarnings('ignore')
one_filter_dragonn = get_SequenceDNN(one_filter_dragonn_parameters)

## Training a DragoNN model

Next, we train the one_filter_dragonn by calling train_SequenceDNN with one_filter_dragonn and simulation_data as the inputs. In each epoch, the one_filter_dragonn will perform a complete pass over the training data, and update its parameters to minimize the loss, which quantifies the error in the model predictions. After each epoch, the code prints performance metrics for the one_filter_dragonn on the validation data. Training stops once the loss on the validation stops improving for multiple consecutive epochs. The performance metrics include balanced accuracy, area under the receiver-operating curve ([auROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)), are under the precision-recall curve ([auPRC](https://en.wikipedia.org/wiki/Precision_and_recall)), area under the precision-recall-gain curve ([auPRG](https://papers.nips.cc/paper/5867-precision-recall-gain-curves-pr-analysis-done-right.pdf)), and recall for multiple false discovery rates  (Recall at [FDR](https://en.wikipedia.org/wiki/False_discovery_rate)).

In [ ]:
train_SequenceDNN(one_filter_dragonn, simulation_data)

We can see that the validation loss is not decreasing and the auROC metric is not decreasing, which indicates this model is not learning. A simple plot of the learning curve, showing the loss function on the training and validation data over the course of training, demonstrates this visually:

In [ ]:
SequenceDNN_learning_curve(one_filter_dragonn)

# A multi-filter DragoNN model 
Next, we modify the model to have 15 convolutional filters instead of just one filter. Will the model learn now?

In [ ]:
multi_filter_dragonn_parameters = {
    'seq_length': 500,
    'num_filters': [15], ## notice the change from 1 filter to 15 filters
    'conv_width': [45],
    'pool_width': 45,
    'dropout': 0.1}
multi_filter_dragonn = get_SequenceDNN(multi_filter_dragonn_parameters)
train_SequenceDNN(multi_filter_dragonn, simulation_data)
SequenceDNN_learning_curve(multi_filter_dragonn)

## Interpreting a DragoNN model using filter visualization
We can see that this model has not learned much because the validation loss has hardly decreased over the course of training and the auROC is only 0.586. Let's see what the sequence filters of this model look like.

In [ ]:
interpret_SequenceDNN_filters(multi_filter_dragonn, simulation_data)

As can be expected, the sequence filters don't reveal patterns that resemble the simulated motifs. Next we explore methods to interpret specific sequences with this DragoNN model.

# Interpreting data with a DragoNN model

Using in-silico mutagenesis (ISM) and [DeepLIFT](https://arxiv.org/pdf/1605.01713v2.pdf), we can obtain scores for specific sequence indicating the importance of each position in the sequence. To assess these methods we compare ISM and DeepLIFT scores to motif scores for each simulated motif at each position in the sequence. These motif scores represent the "ground truth" importance of each position because they are based on the motifs used to simulate the data. We plot provide comaprisons for a positive class sequence on the left and a negative class sequence on the right.

In [ ]:
interpret_data_with_SequenceDNN(multi_filter_dragonn, simulation_data)

We can see that neither DeepLIFT nor ISM highlight the locations of the simulated motifs (highlighted in grey). This is expected because this model doesn't perform well on this simulation.

# A multi-layer DragoNN model
Next, we extend modify multi_filter_dragon to have 3 convolutional layers, with convolutional filter of 15 in each layer, to learn the heterodimer grammar compositionally across multiple layers.

In [ ]:
multi_layer_dragonn_parameters = {
    'seq_length': 500,
    'num_filters': [15, 15, 15], ## notice the change to multiple filter values, one for each layer
    'conv_width': [25, 25, 25], ## convolutional filter width has been modified to 25 from 45
    'pool_width': 45,
    'dropout': 0.1}
multi_layer_dragonn = get_SequenceDNN(multi_layer_dragonn_parameters)
train_SequenceDNN(multi_layer_dragonn, simulation_data)
SequenceDNN_learning_curve(multi_layer_dragonn)

The multi-layered DragoNN model achieves a higher auROC and a lower training and validation loss than the multi-filter DragoNN model. Try the same model without dropout regularization: how important is dropout?

Let's see what the model learns in its sequence filters.

In [ ]:
interpret_SequenceDNN_filters(multi_layer_dragonn, simulation_data)

The sequence filters here are not amenable to interpretation based on visualization alone. In multi-layered models, sequence features are learned compositionally across the layers. As a result, sequence filters in the first layer focus more on simple features that can be combined in higher layers to learn motif features more efficiently, and their interpretation becomes less clear based on simple visualizations. Let's see where ISM and DeepLIFT get us with this model.

In [ ]:
interpret_data_with_SequenceDNN(multi_layer_dragonn, simulation_data)

DeepLIFT and ISM scores for this model on representative positive (left) and negative (right) sequences expose what the model is doing.. The SIX5-ZNF143 grammar is clearly highlighted by both methods in the positive class sequence. However, ISM assigns higher scores to false features around position 250, so we would not be able to distinguish between flase and true features in this example based on ISM score magnitude. DeepLIFT, on the other hand, assigns the highest scores to the true features and therefore it could be used in this example to detect the SIX5-ZNF143 grammar.

# Using DragoNN on your own non-simulated data

The dragonn package provides a command-line interface to train and test DragoNN models, and use them to predict and interpret new data. We start by training a dragonn model on positive and negative sequence:

#### important: If you are running this notebook on the Kundaje Lab public AWS image, please click on Kernel -> Restart in the menu at the top before running the commands below. 
(This will prevent process contention for GPU's)

In [ ]:
!dragonn train --pos-sequences example_pos_sequences.fa --neg-sequences example_neg_sequences.fa --prefix training_example

Based on the provided prefix, this command stores a model file, training_example.model.json, with the model architecture and a weights file, training_example.weights.hd5, with the parameters of the trained model. We test the model by running:

In [ ]:
!dragonn test --pos-sequences example_pos_sequences.fa --neg-sequences example_neg_sequences.fa \
--arch-file training_example.arch.json --weights-file training_example.weights.h5

This command prints the model's test performance metrics on the provided data. Model predictions on sequence data can be obtained by running:

In [ ]:
!dragonn predict --sequences example_pos_sequences.fa --arch-file training_example.arch.json \
--weights-file training_example.weights.h5 --output-file example_predictions.txt

This command stores the model predictions for sequences in example_pos_sequences.fa in the output file example_predictions.txt. We can interpret sequence data with a dragonn model by running:

In [ ]:
!dragonn interpret --sequences example_pos_sequences.fa --arch-file training_example.arch.json \
--weights-file training_example.weights.h5 --prefix example_interpretation

This will write the most important subsequence in each input sequence along with its location in the input sequence in the file example_interpretation.task_0.important_sequences.txt. Note: by default, only examples with predicted positive class probability >0.5 are interpreted. Examples below this threshold yield important subsequence of Ns with location -1. Let's look the first few lines of this file:

In [ ]:
!head example_interpretation.task_0.important_sequences.txt

## Extras for HW

The tutorial example here touches on general principles of DragoNN model development and interpretation. To gain a deeper insight into the difference between DeepLIFT and ISM for model interpretation, consider the following exercise:

Train, test, and run sequence-centric interpretation for the one layered CNN model used here for the following
simulations:
    1. single motif detection simulation of TAL1 in 1000bp sequence with 40% GC content
    (run print_simulation_info("simulate_single_motif_detection") to see the exact simulation parameters)
    2. motif density localization simulation of 2-4 TAL1 motif instances in the central of 150bp of a total 1000bp
    sequence with 40% GC
    content
    (run print_simulation_info("simulate_motif_density_localization") to see the exact simulation parameters)

Key questions:

    1) What could explain the difference in ISM's sensitivity to the TAL1 motif sequence between the simulations?
    2) What does that tell us about the the scope of ISM for feature discovery? Under what conditions is it likely
    to show sensitivity to sequence features?
    
Starter code is provided below to get the data for each simulation and new DragoNN model.


In [ ]:
single_motif_detection_simulation_parameters = {
    "motif_name": "TAL1_known4",
    "seq_length": 1000,
    "num_pos": 10000,
    "num_neg": 10000,
    "GC_fraction": 0.4}

density_localization_simulation_parameters = {
    "motif_name": "TAL1_known4",
    "seq_length": 1000,
    "center_size": 150,
    "min_motif_counts": 2,
    "max_motif_counts": 4,
    "num_pos": 10000,
    "num_neg": 10000,
    "GC_fraction": 0.4}

single_motif_detection_simulation_data = get_simulation_data(
    "simulate_single_motif_detection", single_motif_detection_simulation_parameters)

density_localization_simulation_data = get_simulation_data(
    "simulate_motif_density_localization", density_localization_simulation_parameters)

In [ ]:
new_dragonn_model = get_SequenceDNN(multi_layer_dragonn_parameters)